In [38]:
from enum import Enum 

class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"

In [39]:
import pydriller
from collections import defaultdict
import tqdm
from pathlib import Path
from git import Repo

path_to_repo = "./mastodon"
repo_path = Path('./mastodon')

start_tag =  'v3.5.0'
current_tag = 'v3.5.3'
file_type = '.rb'


if repo_path.exists():
    print('Repository exists!')
else:
    # Clone the repository
    #  clone using the python git library
    Repo.clone_from('https://github.com/mastodon/mastodon', path_to_repo)

repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))

files_by_author = defaultdict(set)
authors_per_file = defaultdict(set)
for commit in commits:
    for modified_file in commit.modified_files:
        author = commit.author.name.strip()
        file = str(modified_file.new_path)
        
        if not file:
            continue
            
        if not file.endswith(file_type):
            continue
        
        files_by_author[author].add(file)        
        authors_per_file[file].add(author)
    
    progress.update(1)


Repository exists!


 98%|█████████▊| 210/214 [00:02<00:00, 83.66commit/s]

In [40]:
# Number of total contributors

print("Total number of contributors: {}".format(len(files_by_author)))


print("Author with most contributions: {}".format(max(files_by_author, key=lambda x: len(files_by_author[x]))), "with {} contributions".format(len(files_by_author[max(files_by_author, key=lambda x: len(files_by_author[x]))])))

Total number of contributors: 11
Author with most contributions: Eugen Rochko with 127 contributions


In [75]:
files_by_author_count = { author : len(files_by_author[author]) for author in files_by_author}
#  sort
files_by_author_count = {k: v for k, v in sorted(files_by_author_count.items(), key=lambda item: item[1], reverse=True)}
files_by_author_count




{'Eugen Rochko': 127,
 'Claire': 51,
 'Holger': 3,
 '0x2019': 3,
 'Jeong Arm': 3,
 'Yamagishi Kazutoshi': 2,
 'Stefano Pigozzi': 2,
 'rinsuki': 1,
 'dogelover911': 1,
 'Sara Golemon': 1,
 'luzpaz': 1}

In [76]:
authors_per_file_count = { file : len(authors_per_file[file]) for file in authors_per_file}
#  sort 
authors_per_file_count = {k: v for k, v in sorted(authors_per_file_count.items(), key=lambda item: item[1], reverse=True)}
authors_per_file_count


{'app/chewy/statuses_index.rb': 3,
 'app/lib/feed_manager.rb': 3,
 'app/helpers/application_helper.rb': 3,
 'lib/mastodon/version.rb': 2,
 'app/helpers/formatting_helper.rb': 2,
 'app/models/status.rb': 2,
 'config/application.rb': 2,
 'app/services/remove_status_service.rb': 2,
 'app/mailers/user_mailer.rb': 2,
 'app/services/activitypub/fetch_featured_collection_service.rb': 2,
 'app/services/activitypub/process_status_update_service.rb': 2,
 'config/environments/production.rb': 2,
 'app/lib/activitypub/activity/create.rb': 2,
 'db/schema.rb': 2,
 'app/services/resolve_account_service.rb': 2,
 'config/initializers/stoplight.rb': 2,
 'app/lib/activitypub/activity/announce.rb': 2,
 'app/serializers/manifest_serializer.rb': 1,
 'config/initializers/paperclip.rb': 1,
 'app/serializers/rest/status_serializer.rb': 1,
 'app/controllers/api/v1/trends/tags_controller.rb': 1,
 'app/controllers/api/v1/admin/account_actions_controller.rb': 1,
 'app/controllers/api/v1/admin/accounts_controller.rb

In [42]:
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.models import ColumnDataSource, CategoricalColorMapper, PrintfTickFormatter, NumeralTickFormatter, Legend, CDSView, GroupFilter, CustomJS, BoxSelectTool, FactorRange
from bokeh.layouts import gridplot, column
from bokeh.models.widgets import Div
from bokeh.palettes import Spectral6, Pastel1, Category20c, Inferno256
import math
import os

output_notebook()

Loading BokehJS ...

In [43]:
# visualize the distribution of authors among files: the visualization should have
# on the x axis the number of authors per file (from 1 to max), and on the y axis
# the number of files with the given number of authors (so for example the first
# bar represent the number of files with single author)

import numpy as np

# we only need a list where every item represents a file, and its value is the number of authors of that file
files_authors = list(authors_per_file_count.values())
max_authors = max(files_authors)

# we create an histogram with numpy, specifying the bins explicitly (instead of automatically) to have round values
hist_files_authors = np.histogram(files_authors, bins=range(1,max_authors + 2))
p = figure(toolbar_location=None, title="Distribution of authors among files", height=300, width=600)
p.vbar(x=hist_files_authors[1][:-1], top=hist_files_authors[0], line_color="white")

# # styling
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.x_range.start = 0.5
p.yaxis.minor_tick_line_color=None
p.xaxis.minor_tick_line_color=None
p.xaxis.ticker = list(range(1,9))
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

show(p)

In [44]:
# visualize the distribution of files among authors: the visualization should have
# on the x axis the number of files per author (from 1 to max), and on the y axis
# the number of authors that own the given number of files (so for example the first
# bar represent the minor contributors, i.e., the number of authors who own 1 file)


# we only need a list where every item represents an author, and its value is the number of files modified by that author
authors_file = list(files_by_author_count.values())
max_files = max(authors_file)

hist_authors_file = np.histogram(authors_file, bins=10)
hist_authors_file

(array([9, 0, 0, 1, 0, 0, 0, 0, 0, 1]),
 array([  1. ,  13.6,  26.2,  38.8,  51.4,  64. ,  76.6,  89.2, 101.8,
        114.4, 127. ]))

In [45]:
bins = [round(x) for x in hist_authors_file[1]]

hist_authors_file = np.histogram(authors_file, bins=bins)
hist_authors_file

(array([9, 0, 0, 0, 1, 0, 0, 0, 0, 1]),
 array([  1,  14,  26,  39,  51,  64,  77,  89, 102, 114, 127]))

In [46]:
bins_str = []
for index, el in enumerate(hist_authors_file[1][:-1]):
    bins_str.append(f"[{el},{hist_authors_file[1][index+1]})")
bins_str

['[1,14)',
 '[14,26)',
 '[26,39)',
 '[39,51)',
 '[51,64)',
 '[64,77)',
 '[77,89)',
 '[89,102)',
 '[102,114)',
 '[114,127)']

In [47]:
p = figure(x_range=bins_str, toolbar_location=None, title="Distribution of files among authors", height=300, width=600)
p.vbar(x=bins_str, top=hist_authors_file[0], line_color="white")

# styling
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis.minor_tick_line_color=None
p.xaxis.minor_tick_line_color=None

show(p)

In [48]:
class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"
    
def calculate_author_contributions(repo, package, method):
    commits = list(repo.traverse_commits())
    progress = tqdm.tqdm(unit="commit", total=len(commits))
    
    result = defaultdict(int)
    for commit in commits:
        for modified_file in commit.modified_files:
            author = commit.author.name.strip()
            file = str(modified_file.new_path)
            
            if not file:
                continue
            
            # skip files outside the queried package
            if not package in file:
                continue
            
            if method == Modification.ADDED:
                result[author] += modified_file.added_lines
            elif method == Modification.REMOVED:
                result[author] += modified_file.deleted_lines
            elif method == Modification.TOTAL:
                result[author] += modified_file.added_lines + modified_file.deleted_lines
            elif method == Modification.DIFF:
                result[author] += modified_file.added_lines - modified_file.deleted_lines
        
        progress.update(1)

    return result

calculate_author_contributions(repo, "app/", Modification.ADDED)

100%|██████████| 214/214 [00:03<00:00, 69.61commit/s]


defaultdict(int,
            {'Claire': 311,
             'Holger': 3,
             'CommanderRoot': 3,
             'rinsuki': 4,
             'Eugen Rochko': 6821,
             '0x2019': 8,
             'dogelover911': 1,
             'Jeong Arm': 8,
             'Chris Dzombak': 1,
             'mayaeh': 23,
             'Sara Golemon': 1,
             'Stefano Pigozzi': 6,
             'Yamagishi Kazutoshi': 17})

In [49]:
method = Modification.TOTAL

# TODO

data_p1 = calculate_author_contributions(repo, "app/", method)
data_p2 = calculate_author_contributions(repo, "lib/", method)
data_p3 = calculate_author_contributions(repo, "public/", method)

100%|██████████| 214/214 [00:02<00:00, 74.20commit/s]


In [50]:
data_p1

defaultdict(int,
            {'Claire': 421,
             'Holger': 6,
             'CommanderRoot': 6,
             'rinsuki': 4,
             'Eugen Rochko': 11242,
             '0x2019': 12,
             'dogelover911': 2,
             'Jeong Arm': 10,
             'Chris Dzombak': 2,
             'mayaeh': 37,
             'Sara Golemon': 2,
             'Stefano Pigozzi': 12,
             'Yamagishi Kazutoshi': 24})

In [51]:
def plot_author_contribution(data, title):
    sorted_data = sorted(data.items(), key=lambda item: item[1], reverse=True)
    x_range = [item[0] for item in sorted_data]
    top = [item[1] for item in sorted_data]

    p = figure(x_range=x_range, 
               height=350, title=title,
               toolbar_location=None, tools="")
    p.vbar(x=x_range, top=top, width=0.9)

    # styling
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    p.y_range.start = 0
    p.yaxis.minor_tick_line_color=None
    p.xaxis.minor_tick_line_color=None
    # p.xaxis.ticker = list(range(1,9))
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

    show(p)

In [52]:
plot_author_contribution(data_p1, "Author Contributions Search (Package 1)")
plot_author_contribution(data_p2, "Author Contributions Index (Package 2)")
plot_author_contribution(data_p3, "Author Contributions Action (Package 3)")


## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder for all dependencies. You task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should
* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`) to serve the html file (necessary for d3 to work)
* open the served `knowledge_loss.html` and look at the visualization

For the package you identify as the worst in terms of knowledge loss, investigate the author contributions using the function defined in the previous exercise and comment how the situation is, e.g. how big the gap between the main author and the second biggest contributor for the selected package is.

In [53]:
repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

method = Modification.TOTAL

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))

files_authors = defaultdict(dict)
for commit in commits:
    for modified_file in commit.modified_files:
        author = commit.author.name.strip()
        file = str(modified_file.new_path)

        if not file:
            continue

        if not file.endswith(file_type):
            continue

        if author not in files_authors[file]:
            files_authors[file][author] = 0
            
        if method == Modification.ADDED:
            files_authors[file][author] += modified_file.added_lines
        elif method == Modification.REMOVED:
            files_authors[file][author] += modified_file.deleted_lines
        elif method == Modification.TOTAL:
            files_authors[file][author] += modified_file.added_lines + modified_file.deleted_lines
        elif method == Modification.DIFF:
            files_authors[file][author] += modified_file.added_lines - modified_file.deleted_lines
        
    
    
    progress.update(1)

files_authors

100%|██████████| 214/214 [00:16<00:00, 13.15commit/s]


defaultdict(dict,
            {'lib/mastodon/version.rb': {'Eugen Rochko': 6, 'Claire': 2},
             'app/chewy/statuses_index.rb': {'Claire': 2,
              'Jeong Arm': 5,
              'Eugen Rochko': 4},
             'app/helpers/formatting_helper.rb': {'Claire': 1,
              'Eugen Rochko': 28},
             'app/lib/feed_manager.rb': {'Claire': 23,
              'dogelover911': 2,
              'Eugen Rochko': 6},
             'app/models/status.rb': {'Claire': 13, 'Jeong Arm': 2},
             'app/serializers/manifest_serializer.rb': {'Holger': 6},
             'config/application.rb': {'Holger': 1, 'Eugen Rochko': 5},
             'config/initializers/paperclip.rb': {'Holger': 20},
             'app/helpers/application_helper.rb': {'Claire': 7,
              'Stefano Pigozzi': 2,
              'Eugen Rochko': 4},
             'app/serializers/rest/status_serializer.rb': {'rinsuki': 4},
             'app/controllers/api/v1/trends/tags_controller.rb': {'Eugen Rochko': 

In [54]:
files_authors

defaultdict(dict,
            {'lib/mastodon/version.rb': {'Eugen Rochko': 6, 'Claire': 2},
             'app/chewy/statuses_index.rb': {'Claire': 2,
              'Jeong Arm': 5,
              'Eugen Rochko': 4},
             'app/helpers/formatting_helper.rb': {'Claire': 1,
              'Eugen Rochko': 28},
             'app/lib/feed_manager.rb': {'Claire': 23,
              'dogelover911': 2,
              'Eugen Rochko': 6},
             'app/models/status.rb': {'Claire': 13, 'Jeong Arm': 2},
             'app/serializers/manifest_serializer.rb': {'Holger': 6},
             'config/application.rb': {'Holger': 1, 'Eugen Rochko': 5},
             'config/initializers/paperclip.rb': {'Holger': 20},
             'app/helpers/application_helper.rb': {'Claire': 7,
              'Stefano Pigozzi': 2,
              'Eugen Rochko': 4},
             'app/serializers/rest/status_serializer.rb': {'rinsuki': 4},
             'app/controllers/api/v1/trends/tags_controller.rb': {'Eugen Rochko': 

In [55]:
contribs_data = {}

for file, author_contribs in files_authors.items():
    total_contribs = sum(author_contribs.values())
    current_percentages = {}
    main_author = None
    max_author_percentage = 0
    if total_contribs != 0:
        for author, contribs in author_contribs.items():
            author_percentage = contribs / total_contribs
            if author_percentage > max_author_percentage:
                max_author_percentage = author_percentage
                main_author = author
        
        contribs_data[file] = {
            "main_author": main_author,
            "main_author_percentage": round(max_author_percentage, 2),
            "total_contribs": total_contribs
        }
        
        
contribs_data

{'lib/mastodon/version.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.75,
  'total_contribs': 8},
 'app/chewy/statuses_index.rb': {'main_author': 'Jeong Arm',
  'main_author_percentage': 0.45,
  'total_contribs': 11},
 'app/helpers/formatting_helper.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.97,
  'total_contribs': 29},
 'app/lib/feed_manager.rb': {'main_author': 'Claire',
  'main_author_percentage': 0.74,
  'total_contribs': 31},
 'app/models/status.rb': {'main_author': 'Claire',
  'main_author_percentage': 0.87,
  'total_contribs': 15},
 'app/serializers/manifest_serializer.rb': {'main_author': 'Holger',
  'main_author_percentage': 1.0,
  'total_contribs': 6},
 'config/application.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.83,
  'total_contribs': 6},
 'config/initializers/paperclip.rb': {'main_author': 'Holger',
  'main_author_percentage': 1.0,
  'total_contribs': 20},
 'app/helpers/application_helper.rb': {'main_aut

## Output Format for Visualization

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.9,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
```

## JSON Export

For exporting the data to JSON you can use the following snippet:

```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

In [56]:
main_authors = defaultdict(int)
for info in contribs_data.values():
    main_authors[info['main_author']] += 1
    
sorted(main_authors.items(), key=lambda k: k[1], reverse=True)

[('Eugen Rochko', 122),
 ('Claire', 43),
 ('0x2019', 3),
 ('Holger', 2),
 ('Jeong Arm', 1),
 ('rinsuki', 1),
 ('Sara Golemon', 1),
 ('Stefano Pigozzi', 1),
 ('luzpaz', 1)]

In [57]:
main_author_by_contribs = defaultdict(int)
for file, author_contribs in files_authors.items():
    for author, contribs in author_contribs.items():
        main_author_by_contribs[author] += contribs

sorted(main_author_by_contribs.items(), key=lambda k: k[1], reverse=True)        

[('Eugen Rochko', 2308),
 ('Claire', 934),
 ('Holger', 27),
 ('0x2019', 13),
 ('Jeong Arm', 9),
 ('Stefano Pigozzi', 4),
 ('rinsuki', 4),
 ('Yamagishi Kazutoshi', 4),
 ('luzpaz', 4),
 ('dogelover911', 2),
 ('Sara Golemon', 2)]

In [58]:
main_author = "uboness"


def add_to_tree(branch, tree, info):
    
    if len(branch) == 1:
        if info["main_author"] == main_author:
            color = "red"
        else:
            color = "navy"
                        
        tree["children"].append({
            "author_color": color,
            "size": info["total_contribs"],
            "weight": info["total_contribs"],
            "name": branch[0],
            "ownership": info["main_author_percentage"],
            "children": []
        })
    else:
        node = branch[0]
        others = branch[1:]
        
        child = None
        for c in tree["children"]:
            if c["name"] == node:
                child=c
                break                
        
        if not child:
            child = {
                "name": node,
                "children": []
            }
            tree["children"].append(child)
            
        add_to_tree(others, child, info)
        
tree = {
    "name": "root",
    "children": []
}

for file, info in contribs_data.items():
    parts = file.split("/")
    add_to_tree(parts, tree, info)


tree
import json

with open("output-2022.json", "w") as file:
    json.dump(tree, file, indent=4)

In [59]:
'app/account/models/account.rb'.split('/')

['app', 'account', 'models', 'account.rb']

In [60]:
data_bucket = calculate_author_contributions(repo, "app/accounts", Modification.TOTAL)
plot_author_contribution(data_bucket, "Author contributions for app/accounts")

100%|██████████| 214/214 [00:03<00:00, 70.63commit/s]


## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project. Visualize the code churn over time bucketing the data by day. Remember that you'll need to interpolate the data for days when there are no commits. Chose an interpolation strategy and justify it.

Look at the churn trend over time and identify two outliers. For each of them:
- identify if it was caused by a single or multiple commits (since you are bucketing the data by day)
- find the hash of the involved commit(s)
- find the involved files
- look at the actual diff

Based on the above, discuss if the outlier is a false positive or should be a reason for concern.

In [61]:
import datetime
import pytz
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.models import ColumnDataSource, CategoricalColorMapper, PrintfTickFormatter, NumeralTickFormatter, Legend, CDSView, GroupFilter, CustomJS, BoxSelectTool, FactorRange
from bokeh.layouts import gridplot, column
from bokeh.models.widgets import Div
from bokeh.palettes import Spectral6, Pastel1, Category20c, Inferno256
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.plotting import figure, show
from dateutil.rrule import rrule, DAILY


repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))
timeseries_a = defaultdict(int)
timeseries_d = defaultdict(int)
timeseries_t = defaultdict(int)

for commit in commits:
    timeseries_a[commit.committer_date.date()] += commit.insertions
    timeseries_d[commit.committer_date.date()] += commit.deletions
    timeseries_t[commit.committer_date.date()] += commit.insertions + commit.deletions
    progress.update(1)
    

 97%|█████████▋| 208/214 [00:05<00:00, 47.61commit/s]

In [62]:
timeseries_a

defaultdict(int,
            {datetime.date(2022, 3, 30): 7,
             datetime.date(2022, 3, 31): 16,
             datetime.date(2022, 4, 1): 26,
             datetime.date(2022, 4, 3): 1,
             datetime.date(2022, 4, 4): 8,
             datetime.date(2022, 4, 5): 4,
             datetime.date(2022, 4, 6): 505,
             datetime.date(2022, 4, 7): 60,
             datetime.date(2022, 4, 8): 1383,
             datetime.date(2022, 4, 9): 5,
             datetime.date(2022, 4, 11): 1,
             datetime.date(2022, 4, 12): 2,
             datetime.date(2022, 4, 13): 1,
             datetime.date(2022, 4, 14): 573,
             datetime.date(2022, 4, 16): 18,
             datetime.date(2022, 4, 19): 2,
             datetime.date(2022, 4, 22): 280,
             datetime.date(2022, 4, 21): 1,
             datetime.date(2022, 4, 23): 8,
             datetime.date(2022, 4, 24): 167,
             datetime.date(2022, 4, 26): 82,
             datetime.date(2022, 4, 27): 134,
     

In [63]:
def extrapolate_ts_day(timeseries, start_date, end_date):
    extrapolated_ts = []
    for dt in rrule(DAILY, dtstart=start_date, until=end_date):
        value = timeseries.get(dt.date(), 0)
        extrapolated_ts.append([dt, value])
    return extrapolated_ts


first_commit_date = commits[0].committer_date
last_commit_date = commits[-1].committer_date
full_timeseries_a = extrapolate_ts_day(timeseries_a, first_commit_date, last_commit_date)
full_timeseries_d = extrapolate_ts_day(timeseries_d, first_commit_date, last_commit_date)
full_timeseries_t = extrapolate_ts_day(timeseries_t, first_commit_date, last_commit_date)


In [64]:
full_timeseries_a

[[datetime.datetime(2022, 3, 30, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  7],
 [datetime.datetime(2022, 3, 31, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  16],
 [datetime.datetime(2022, 4, 1, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  26],
 [datetime.datetime(2022, 4, 2, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  0],
 [datetime.datetime(2022, 4, 3, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  1],
 [datetime.datetime(2022, 4, 4, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  8],
 [datetime.datetime(2022, 4, 5, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  4],
 [datetime.datetime(2022, 4, 6, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc7cd0>),
  505],
 [datetime.datetime(2022, 4, 7, 14, 52, 37, tzinfo=<git.objects.util.tzoffset object at 0x7efe7ebc

In [65]:
data = {
    "day": [x[0] for x in full_timeseries_a],
    "addition": [x[1] for x in full_timeseries_a],
    "deletion": [x[1] for x in full_timeseries_d],
    "total": [x[1] for x in full_timeseries_t]    
}

source = ColumnDataSource(data)

p = figure(height=300, width=980, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(data["day"][1], data["day"][10]))

hover = HoverTool(
    tooltips=[
        ("Day", "@day{%F}"),
        ("Lines added", "@addition{0,0}"),
        ("Lines removed", "@deletion{0,0}"),
        ("Lines total", "@total{0,0}")        
    ],
    formatters={
        '@day': 'datetime' # use 'datetime' formatter for '@date' field
    },

    mode='vline',
#     names=["addition"]
)

p.line('day', 'addition', source=source, name="addition", line_width=2)
p.line('day', 'deletion', source=source, line_color="red", line_width=2)
p.line('day', 'total', source=source, line_color="black", line_width=2)
p.add_tools(hover)
p.yaxis.axis_label = 'Churn'

select = figure(title="Zoom",
                height=130, width=980, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('day', 'addition', source=source)
select.line('day', 'deletion', source=source, line_color="red")
select.line('day', 'total', source=source, line_color="black")
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))


In [66]:
spike1 = [c for c in commits if c.committer_date.date() == datetime.date(2022,2,26)]
[c.hash for c in spike1]

[]

In [67]:
def print_diff(commits):
    additions = defaultdict(int)
    deletions = defaultdict(int)
    for commit in commits:
        for f in commit.modified_files:
            additions[str(f.new_path)] += f.added_lines
            deletions[str(f.new_path)] += f.deleted_lines

    all_files = set()
    all_files.update(additions.keys())
    all_files.update(deletions.keys())

    for f in sorted(all_files):
        print(f"+{additions.get(f,0)}\t -{deletions.get(f,0)} \t {f}")

In [68]:
print_diff(spike1)

In [69]:
spike2 = [c for c in commits if c.committer_date.date() == datetime.date(2022,2,11)]
[c.hash for c in spike2]

[]

In [70]:
print_diff(spike2)

In [71]:
for commit in spike2:
    for f in commit.modified_files:
        if f.new_path is None:
            print(f.old_path)

In [72]:
for commit in spike2:
    print(commit.msg)
    print("================================================================")